In [ ]:
import os
from osgeo import ogr

data_dir = r'osgeopy-data'

##  Overlay tools: what’s on top of what?

### Study area of interes

In [ ]:
# Look at New Orleans wetlands. First get a specific marsh feature near New Orleans.
water_ds = ogr.Open(os.path.join(data_dir, 'us', 'wtrbdyp010.shp'))
water_lyr = water_ds.GetLayer(0)
water_lyr.SetAttributeFilter('WaterbdyID = 1011327')
marsh_feat = water_lyr.GetNextFeature()
marsh_geom = marsh_feat.geometry().Clone()

In [ ]:
# Get the New Orleans boundary.
nola_ds = ogr.Open(os.path.join(data_dir, 'louisiana', 'NOLA.shp'))
nola_lyr = nola_ds.GetLayer(0)
nola_feat = nola_lyr.GetNextFeature()
nola_geom = nola_feat.geometry().Clone()

In [ ]:
# Intersect the marsh and boundary polygons to get the part of the marsh that
# falls within New Orleans city boundaries.
intersection = marsh_geom.Intersection(nola_geom)

### Figure out how much of New Orleans is wetlands

#### 1. Using individual geometries

In [ ]:
# Throw out lakes and anything not in the vicinity of New Orleans,and then loop through the remaining water body features.
water_lyr.SetAttributeFilter("Feature != \'Lake\'")
water_lyr.SetSpatialFilter(nola_geom)

In [ ]:
# For each one, find the area of the feature that is contained within city boundaries and add it to a running total.
wetlands_area = 0
for feat in water_lyr:
    intersect = feat.geometry().Intersection(nola_geom)
    wetlands_area += intersect.GetArea()

In [ ]:
# Then it's easy to figure the percentage by dividing that total by the area of New Orleans.
pcnt = wetlands_area / nola_geom.GetArea()
print('{:.1%} of New Orleans is wetland'.format(pcnt))

#### 2. Using layers instead of individual geometries

In [ ]:
#You need to set the attribute filter, but a spatial filter isn't necessary
water_lyr.SetSpatialFilter(None)
water_lyr.SetAttributeFilter("Feature != \'Lake\'")

In [ ]:
# In this case you'll need an empty layer to store the intersection results in, so create a temporary one in memory
memory_driver = ogr.GetDriverByName('Memory')
temp_ds = memory_driver.CreateDataSource('temp')
temp_lyr = temp_ds.CreateLayer('temp')

In [ ]:
# Then run the intersection and use SQL to sum up the areas.
nola_lyr.Intersection(water_lyr, temp_lyr)
sql = 'SELECT SUM(OGR_GEOM_AREA) AS area FROM temp'
lyr = temp_ds.ExecuteSQL(sql)
pcnt = lyr.GetFeature(0).GetField('area') / nola_geom.GetArea()
print('{:.1%} of New Orleans is wetland'.format(pcnt))

## Proximity tools: how far apart are things?

### 1. Let’s figure out how many cities in the United States are within 10 miles of a volcano. 
   * We’ll use datasets that have an Albers projection so that the map units are meters instead of decimal degrees
   * We’ll also use this example to highlight a potential source of error when doing analyses like this
   * The first step in your analysis will be to buffer a volcano dataset by 16,000 meters, which is roughly equivalent to 10 miles
   * Because there isn’t a buffer function on an entire layer, you’ll buffer each volcano point individually and add it to a temporary layer
   * Once that’s done, you can intersect the buffer layer with the cities layer to get the number of cities that fall
within that 10-mile radius

#### 1. A flawed method for determining the number of cities near volcanoes

In [ ]:
# Open layers for examples.
shp_ds = ogr.Open(r'osgeopy-data\us')
volcano_lyr = shp_ds.GetLayer('us_volcanos_albers')
cities_lyr = shp_ds.GetLayer('cities_albers')

In [ ]:
# Make a temporary layer to store buffers
memory_driver = ogr.GetDriverByName('memory')
memory_ds = memory_driver.CreateDataSource('temp')

buff_lyr = memory_ds.CreateLayer('buffer')
buff_feat = ogr.Feature(buff_lyr.GetLayerDefn())

In [ ]:
# Buffer each volcano and add it to the buffer layer
for volcano_feat in volcano_lyr:
    buff_geom = volcano_feat.geometry().Buffer(16000)
    tmp = buff_feat.SetGeometry(buff_geom)
    tmp = buff_lyr.CreateFeature(buff_feat)

In [ ]:
#Intersect the cities and volcano buffers
result_lyr = memory_ds.CreateLayer('result')
buff_lyr.Intersection (cities_lyr, result_lyr)
print('Cities: {}'.format(result_lyr.GetFeatureCount()))

**Note:** From this you could conclude that are **83 cities** in the United States that are within **10 miles of a volcano**. But for good measure, try doing the same thing with the slightly different method.

#### 2. A better method for determining the number of cities near volcanoes
* This time you’ll add the buffers to a multipolygon instead of a temporary layer
* A function called `UnionCascaded()` efficiently unions all of the polygons in a multipolygon together
* You’ll use this to create one polygon from all of the volcano buffers and then use the result as a spatial filter
on the cities layer

In [ ]:
# Go back to the start of the volcanoes layer
volcano_lyr.ResetReading()

multipoly = ogr.Geometry(ogr.wkbMultiPolygon)
for volcano_feat in volcano_lyr:
    buff_geom = volcano_feat.geometry().Buffer(16000)
    multipoly.AddGeometry(buff_geom)

**Note**: Use `UnionCascaded()` when you need to union many geometries together.
It will be much faster than joining them one by one.

In [ ]:
# Unions all of the polygons in a multipolygon together
cities_lyr.SetSpatialFilter(multipoly.UnionCascaded ())
print('Cities: {}'.format(cities_lyr.GetFeatureCount()))

**We' ve lost five cities!!! What happened?**
In the first example, a copy of a city is included in the output every time it falls within a volcano buffer:
 * This means a city will be included more than once if it’s within 16,000 meters of multiple volcanoes
 * This happened with a few cities, which is why the count from the intersection method was wrong, and higher than
from the spatial filter method.

**Note**: This is a good example of why you should always think through your methodology carefully, because the “obvious” solution might be incorrect and provide the wrong results.

### 2. How far a particular city is from a certain volcano?

In [ ]:
# Open layers for examples.
shp_ds = ogr.Open(os.path.join(data_dir, 'us'))
volcano_lyr = shp_ds.GetLayer('us_volcanos_albers')
cities_lyr = shp_ds.GetLayer('cities_albers')

In [ ]:
# Find out how far Seattle is from Mount Rainier.
volcano_lyr.SetAttributeFilter("NAME = 'Rainier'")
feat = volcano_lyr.GetNextFeature()
rainier = feat.geometry().Clone()

cities_lyr.SetSpatialFilter(None)
cities_lyr.SetAttributeFilter("NAME = 'Seattle'")
feat = cities_lyr.GetNextFeature()
seattle = feat.geometry().Clone()

meters = round(rainier.Distance(seattle))
miles = meters / 1600
print('{} meters ({} miles)'.format(meters, miles))


### !!!! 2.5D Geometries: z values aren’t used when performing spatial operations.

In [ ]:
# Take a look at the distance between two 2D points. The distance should be 4.
pt1_2d = ogr.Geometry(ogr.wkbPoint)
pt1_2d.AddPoint(15, 15)
pt2_2d = ogr.Geometry(ogr.wkbPoint)
pt2_2d.AddPoint(15, 19)
print(pt1_2d.Distance(pt2_2d))


In [ ]:
# Now create some 2.5D points, using the same x and y coordinates, but adding z coordinates. 
# The distance now, if three dimensions were taken into account would be 5. But ogr still returns 4. 
# This is because the z coordinates are ignored.
pt1_25d = ogr.Geometry(ogr.wkbPoint25D)
pt1_25d.AddPoint(15, 15, 0)
pt2_25d = ogr.Geometry(ogr.wkbPoint25D)
pt2_25d.AddPoint(15, 19, 3)
print(pt1_25d.Distance(pt2_25d))

In [ ]:
# Take a look at the area of a 2D polygon. The area should be 100.
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(10, 10)
ring.AddPoint(10, 20)
ring.AddPoint(20, 20)
ring.AddPoint(20, 10)
poly_2d = ogr.Geometry(ogr.wkbPolygon)
poly_2d.AddGeometry(ring)
poly_2d.CloseRings()
print(poly_2d.GetArea())

In [ ]:
# Now create a 2.5D polygon, again using the same x and y coordinates, but
# providing a z coordinate for a couple of the vertices. 
# The area of this in three dimensions is around 141, but ogr still returns 100.
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(10, 10, 0)
ring.AddPoint(10, 20, 0)
ring.AddPoint(20, 20, 10)
ring.AddPoint(20, 10, 10)
poly_25d = ogr.Geometry(ogr.wkbPolygon25D)
poly_25d.AddGeometry(ring)
poly_25d.CloseRings()
print(poly_25d.GetArea())

In [ ]:
# If three dimensions were taken into account, pt1_d2 would be contained in the 2D polygon, but not the 3D one. 
# But since the 3D one is really 2.5D, ogr thinks the point is contained in both polygons.
print(poly_2d.Contains(pt1_2d))
print(poly_25d.Contains(pt1_2d))